In [1]:
import keras
from keras.models import Model
from keras.layers import Flatten,Dense,Dropout
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
Dataset_path="/gdrive/My Drive/Colab Notebooks/Untitled folder/data/image_data"

In [36]:
from google.colab import drive
drive.mount('/gdrive')
Dataset_path="/gdrive/My Drive/Colab Notebooks/Untitled folder"
!mkdir "/gdrive/My Drive/Colab Notebooks/Untitled folder/data"
!unzip -q "/gdrive/My Drive/Colab Notebooks/Untitled folder/image_data.zip" -d "/gdrive/My Drive/Colab Notebooks/Untitled folder/data"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
num_classes=5
Batch_size=16
num_epochs=20
weight_final="model-resnet50-final.csv"
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
IMAGE_SIZE = (224, 224)

In [38]:
train_batches = train_datagen.flow_from_directory(Dataset_path + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=Batch_size)

Found 2323 images belonging to 5 classes.


In [39]:
valid_datagen = ImageDataGenerator()
valid_batches = train_datagen.flow_from_directory(Dataset_path + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=Batch_size)

Found 500 images belonging to 5 classes.


In [40]:
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [41]:
FREEZE_LAYERS = 10
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // Batch_size,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // Batch_size,
                        epochs = num_epochs)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [74]:
import keras.preprocessing.image as image
#import tenserflow as tf
import os
import cv2
#list_ds = keras.data.Dataset.list_files(str(Dataset_path + '/test'))
test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_directory(directory=Dataset_path+ "/test_1",batch_size=Batch_size,shuffle=False,class_mode=None,target_size=IMAGE_SIZE)




Found 2000 images belonging to 1 classes.


In [83]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=net_final.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

125/125 [==============================] - 31s 251ms/step


In [87]:
predicted_class_indices=np.argmax(pred,axis=1)

['rose', 'tulip', 'tulip', 'tulip', 'tulip', 'rose', 'dandelion', 'tulip', 'rose', 'tulip', 'tulip', 'dandelion', 'tulip', 'rose', 'daisy', 'daisy', 'sunflower', 'rose', 'dandelion', 'daisy', 'tulip', 'daisy', 'sunflower', 'dandelion', 'tulip', 'sunflower', 'sunflower', 'tulip', 'rose', 'rose', 'rose', 'rose', 'tulip', 'sunflower', 'sunflower', 'sunflower', 'daisy', 'rose', 'rose', 'sunflower', 'rose', 'dandelion', 'sunflower', 'tulip', 'sunflower', 'tulip', 'tulip', 'dandelion', 'rose', 'daisy', 'daisy', 'tulip', 'daisy', 'daisy', 'dandelion', 'sunflower', 'rose', 'daisy', 'sunflower', 'tulip', 'rose', 'rose', 'sunflower', 'daisy', 'rose', 'rose', 'daisy', 'tulip', 'rose', 'tulip', 'rose', 'sunflower', 'dandelion', 'dandelion', 'dandelion', 'sunflower', 'daisy', 'tulip', 'tulip', 'daisy', 'tulip', 'rose', 'sunflower', 'rose', 'tulip', 'sunflower', 'tulip', 'rose', 'daisy', 'tulip', 'dandelion', 'tulip', 'daisy', 'sunflower', 'dandelion', 'tulip', 'daisy', 'rose', 'sunflower', 'sunflow

In [0]:
import pandas as pd
filenames=test_generator.filenames
results=pd.DataFrame({"id":filenames,
                      "flower_class":predicted_class_indices})
results.to_csv(Dataset_path+"results.csv",index=False)